<a href="https://colab.research.google.com/github/adeepH/User_based-Movie-Recommendation/blob/using-RBM/recommenderSystems_RBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
# Importing the dataset
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [0]:
#Preparing the training set and the test set
training_set = pd.read_csv('u1.base',delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('u1.test',delimiter='\t')
test_set = np.array(test_set,dtype='int')

In [0]:
#getting the number of users and movies
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))

In [0]:
#Converting the data into an array with users in lines and movies in columns

def convert(data):
  new_data = []
  for id_users in range(1,nb_users+1):
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,1][data[:,0] == id_users]
    ratings =np.zeros(nb_movies)
    ratings[id_movies -1] = id_ratings
    new_data.append(list(ratings))
  return new_data


training_set = convert(training_set)
test_set = convert(test_set)



In [0]:
#Converting the data into torch tensor
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
#converting the ratings into binary ratings 1(liked) and 0(disliked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
#there is no or function in torch tensors
training_set[training_set >=3] = 1
#now for the test set to be converted into torch tensors
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1





In [0]:
#Creating the architecture of the Neural Network
"""RBM is a probablistic model so we define a class RBm
#self is the object that will be created afterwards
nv is the number of visible nodes and nh being the number of hidden nodes"""
"""defining the biasal distribution"""
#2D tensor
"""bias for visible nodes"""
"""activation function represents the probability"""
#"""refer to the Restricted boltzman machines paper published for the below algorithm"""
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)


In [0]:
#creating an RBM object
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)


In [25]:
#Training the  sample RBM
#We are going to normalise the training set
# to return only one of the 2 things from sample h
#k steps of the contrastic divergence in for loops
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(0,10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))



epoch: 1 loss: tensor(0.1996)
epoch: 2 loss: tensor(0.0105)
epoch: 3 loss: tensor(0.0049)
epoch: 4 loss: tensor(0.0028)
epoch: 5 loss: tensor(0.0020)
epoch: 6 loss: tensor(0.0013)
epoch: 7 loss: tensor(0.0012)
epoch: 8 loss: tensor(0.0011)
epoch: 9 loss: tensor(0.0008)
epoch: 10 loss: tensor(0.0009)


In [29]:
#Testing the RBM

for epoch in range(1, nb_epoch + 1):
    test_loss = 0
    s = 0.
    for id_user in range(0, nb_users):
        v = training_set[id_user:id_user+1]
        vt = test_set[id_user:id_user+1]
        if len(vt[vt>=0]) > 0:
            _,h = rbm.sample_h(v)
            _,v = rbm.sample_v(h)
            test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
            s += 1.
    print('test_loss: '+str(test_loss/s))
#print( ' Accuracy : '+str(1 - test_loss/s)*100 +'%')


test_loss: tensor(0.0024)
test_loss: tensor(0.0021)
test_loss: tensor(0.0024)
test_loss: tensor(0.0021)
test_loss: tensor(0.0020)
test_loss: tensor(0.0020)
test_loss: tensor(0.0023)
test_loss: tensor(0.0023)
test_loss: tensor(0.0018)
test_loss: tensor(0.0021)


In [30]:
#using a different loss function
#Root Mean Squared Error(RMSE)
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(0,10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])**2)
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.0010)
epoch: 2 loss: tensor(0.0006)
epoch: 3 loss: tensor(0.0006)
epoch: 4 loss: tensor(0.0004)
epoch: 5 loss: tensor(0.0004)
epoch: 6 loss: tensor(0.0004)
epoch: 7 loss: tensor(0.0004)
epoch: 8 loss: tensor(0.0004)
epoch: 9 loss: tensor(0.0003)
epoch: 10 loss: tensor(0.0003)


In [33]:
#Testing the RBM using a different Loss function
for epoch in range(1, nb_epoch + 1):
    test_loss_2 = 0
    s = 0.
    for id_user in range(0, nb_users):
        v = training_set[id_user:id_user+1]
        vt = test_set[id_user:id_user+1]
        if len(vt[vt>=0]) > 0:
            _,h = rbm.sample_h(v)
            _,v = rbm.sample_v(h)
            test_loss_2 += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])**2)
            s += 1.
    print('test_loss: '+str(test_loss_2/s))

test_loss: tensor(0.0007)
test_loss: tensor(0.0010)
test_loss: tensor(0.0015)
test_loss: tensor(0.0011)
test_loss: tensor(0.0013)
test_loss: tensor(0.0007)
test_loss: tensor(0.0010)
test_loss: tensor(0.0010)
test_loss: tensor(0.0009)
test_loss: tensor(0.0011)


In [34]:
#comparing the 2 torch tensors
torch.max(test_loss,test_loss_2)

tensor(0.4931)